# POC Model

__Install packages__

In [124]:
import re
import numpy as np
import pandas as pd
from pprint import pprint
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy
import pyLDAvis
import pyLDAvis.gensim
from nltk.corpus import stopwords
import glob
from datetime import datetime
import statistics 
import jmespath
import json
from rake_nltk import Rake
from nltk.tokenize import word_tokenize
from nltk.tokenize import MWETokenizer
from gensim.test.utils import datapath
import math
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process

//anaconda3/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


### Converting Tree to JSON format

In [2]:
# Please run this code cell once only as multiple runs will affect the same file 
with open('Clean_mind_map.json') as json_file:
    data = json.load(json_file)
    
itemsKeyedById = {i["ID"]: i for i in data}

#iterate through each item in the `myJson` list.
for item in data[1:]:
    #does the item have a parent?
    if "Parent" in item:
        #get the parent item
        parent = itemsKeyedById[item['Parent']]
        #if the parent item doesn't have a "children" member, 
        #we must create one.
        if "children" not in parent:
            parent["children"] = []
        #add the item to its parent's "children" list.
        parent["children"].append(item)
topLevelItems = [item for item in data if "Parent" not in item]
taxonomy_data = {'data': data}

In [3]:
with open('data.json', 'w') as outfile:
    json.dump(taxonomy_data, outfile)

Examples are found on this link: http://jmespath.org/tutorial.html

In [4]:
expression = jmespath.compile("data[?Title =='Lean of Peak'].children[].Title")
expression.search(data)

### Data Sources Integration

In [19]:
data = []
for filename in glob.glob('*.csv'):
    data.append(filename)
data

['LDA_vector.csv',
 'COPA_forum_threads.csv',
 'COPA_forum_replies.csv',
 'wiki_formal_total_clean.csv',
 'COPA_Blogs.csv',
 'COPASearches.csv']

In [6]:
wiki = pd.read_csv(data[3])
wiki = wiki[['Date', 'Title', 'Content', 'Author','Like']]
wiki['Author'] = [a.strip() for a in wiki['Author']]
wiki['Comment'] = 0
wiki['Resource'] = 'Wiki'

In [81]:
# magazine = pd.read_csv(data[])
# magazine_date_list = list(magazine['Magazine'].unique())
# # Get the month of magazine from its name
# # Since the the COPA magazines are released monthly, so it only has year and month
# # in its date column. In this case we just assume all magazines were released on the 
# # first day of that month.
# magazine_date_dict = {magazine_date_list[0]: '2015-01-01', magazine_date_list[1]: '2019-01-01',
#                       magazine_date_list[2]: '2015-07-01', magazine_date_list[3]: '2019-04-01',
#                       magazine_date_list[4]: '2019-06-01', magazine_date_list[5]: '2018-01-01',
#                       magazine_date_list[6]: '2017-09-01', magazine_date_list[7]: '2016-09-01',
#                       magazine_date_list[8]: '2017-11-01', magazine_date_list[9]: '2015-05-01',
#                       magazine_date_list[10]: '2015-09-01', magazine_date_list[11]: '2006-09-01', 
#                       magazine_date_list[12]: '2018-06-01', magazine_date_list[13]: '2019-07-01',
#                       magazine_date_list[14]: '2017-04-01', magazine_date_list[15]: '2016-04-01',
#                       magazine_date_list[16]: '2018-11-01', magazine_date_list[17]: '2015-09-01',
#                       magazine_date_list[18]: '2019-03-01', magazine_date_list[19]: '2006-11-01',
#                       magazine_date_list[20]: '2016-01-01', magazine_date_list[21]: '2019-05-01',
#                       magazine_date_list[22]: '2016-03-01', magazine_date_list[23]: '2016-05-01',
#                       magazine_date_list[24]: '2006-07-01', magazine_date_list[25]: '2017-06-01',
#                       magazine_date_list[26]: '2018-07-01', magazine_date_list[27]: '2018-09-01',
#                       magazine_date_list[28]: '2017-01-01', magazine_date_list[29]: '2012-11-01'}
# magazine['Magazine'] = magazine['Magazine'].map(magazine_date_dict)
# magazine.columns = ['Date', 'Title', 'Content', 'Author']
# magazine['Like'] = 0
# magazine['Comment'] = 0
# magazine['Resource'] = 'Magazine'

In [7]:
blog = pd.read_csv(data[4], encoding = 'unicode_escape')
blog = blog[['Date', 'Title', 'Body', 'Author', 'Like','Comment']]
blog.columns = ['Date', 'Title', 'Content', 'Author', 'Like','Comment']
blog['Date'] = [d.split(' ')[0].split('/')[2] + '-' + d.split(' ')[0].split('/')[0] for d in blog['Date']]
blog['Date'] = blog['Date'].map(lambda x: x + '-01')
blog['Resource'] = 'Blog'

In [20]:
forum = pd.read_csv(data[2])
forum = forum[['ThreadReplyDate', 'UserID', 'FormattedBody']]
forum.head()

,ThreadReplyDate,UserID,FormattedBody
0,2008-06-25 10:57:03,3870,You are cordially invited to a Cirrus BBQ & Fu...
1,2005-10-23 00:56:48,4255,"Ed,Thank you for the offer. My envelope is on ..."
2,2005-10-23 02:02:14,3426,If I come over Tuesday do I get to play Igor t...
3,2005-10-24 04:31:25,2296,"Ed, My envelope is on the way also! thanks for..."
4,2005-11-04 01:34:09,3586,"When I returned home from S FL yesterday, I co..."


In [8]:
pdList = [wiki, blog]
# Combine three datasets together
df = pd.concat(pdList)
df.reset_index(drop = True, inplace = True)

In [7]:
#Ranking the source, blog is the most important source so have 3
raw_data = {'Resource': ['Wiki','Blog','Magazine'], 'SourceScore': [1, 2, 3]}
Source_tb = pd.DataFrame(raw_data, columns = ['Resource', 'SourceScore'])
#add source score to the main table
df = pd.merge(df, Source_tb, left_on = 'Resource', right_on = 'Resource')

# Generate Attributes

__Generate Recency Score__

In [8]:
def  Calculate_RecencyScore(date):
    '''
    Recency Rate = log( 1 + 1/(days between the post date and current date + 1))
    '''
    date_datetime = datetime.strptime(date, '%Y-%m-%d').date()
    rececncy_rate = math.log10(1/(((datetime.date(datetime.now()))-date_datetime).days+1)+1)
    return rececncy_rate
    

df['RecencyRate'] = df['Date'].map(lambda y: Calculate_RecencyScore(y))

__Generate Author Score__

In [9]:
def Calculate_AuthorScore(author):
    '''
    AuthorScore = Number of posts of specified content source * pre-determined weight
    '''
    author_score = df[df['Author'] == author]['SourceScore'].sum()
    return author_score

author_list = list(df['Author'].unique())
df['AuthorScore'] = df['Author'].map(lambda x: Calculate_AuthorScore(x), author_list)

__Generate Content Score__

In [21]:
t1 = datetime.now()

forum = pd.read_csv(data[2])
data = forum.FormattedBody.values.tolist()

# Create a model for topic classification based on the dataset we have 
# (It may cause some problems because the model will be applied to the dataset later to give each content a topic.)
# (The problem can be solved when we get the entire dataset since we can take part of the contents as corpus.)

# Get some stop words
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

t2 = datetime.now()
diff = t2 - t1
print (diff.seconds)

# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

t3 = datetime.now()
diff = t3 - t2
print (diff.seconds)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

t4 = datetime.now()
diff = t4 - t3
print (diff.seconds)

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           passes=10)

pprint(lda_model.print_topics(num_words = 10))
# Get 20 related topics
doc_lda = lda_model[corpus]

t5 = datetime.now()
diff = t5 - t4
print (diff.seconds)

917
475
3351
[(0,
  '0.026*"system" + 0.022*"work" + 0.022*"update" + 0.019*"use" + 0.018*"mfd" '
  '+ 0.018*"datum" + 0.018*"display" + 0.017*"software" + 0.014*"avidyne" + '
  '0.013*"avionic"'),
 (1,
  '0.040*"pay" + 0.030*"year" + 0.020*"cost" + 0.019*"company" + '
  '0.018*"insurance" + 0.016*"business" + 0.016*"money" + 0.015*"claim" + '
  '0.014*"tax" + 0.013*"rate"'),
 (2,
  '0.108*"ice" + 0.035*"vote" + 0.028*"energy" + 0.027*"condition" + '
  '0.026*"medical" + 0.020*"visual" + 0.018*"drug" + 0.015*"registration" + '
  '0.013*"race" + 0.013*"image"'),
 (3,
  '0.033*"turn" + 0.024*"mode" + 0.023*"approach" + 0.022*"hold" + '
  '0.020*"course" + 0.020*"head" + 0.019*"switch" + 0.018*"fix" + '
  '0.018*"autopilot" + 0.016*"button"'),
 (4,
  '0.073*"land" + 0.064*"runway" + 0.061*"landing" + 0.046*"airport" + '
  '0.030*"field" + 0.028*"pattern" + 0.027*"wind" + 0.022*"final" + '
  '0.018*"turn" + 0.016*"approach"'),
 (5,
  '0.052*"weight" + 0.044*"law" + 0.043*"carry" + 0.036*"w

In [64]:
cleanlist = []
for d in data:
    try:
        cleanlist.append(d.lower())
    except:
        continue        

In [65]:
# Put contents into model and assign each content to a topic
texts1 = [[word for word in doc.split() if word not in stop_words] for doc in cleanlist]
corpus1 = [id2word.doc2bow(text) for text in texts1]
resultlist = lda_model.get_document_topics(corpus1)

In [66]:
# Build a dictionary containing the index of content and its topic classification
topic_dict = {}
for i, result in enumerate(resultlist):
    temp = 0
    topic = 0
    for j in range(len(result)):
        if result[j][1] > temp:
            temp = result[j][1]
            topic = j
    topic_dict[i] = topic

In [94]:
len(topic_dict)

1034411

# Compare keyword vector with reference vector

__Match replies with searching terms by topic only__

In [304]:
def topic_match(search_query):
    '''
    Extract the topic of searching query by LDA model trained by forum posts.
    '''
    texts1 = [[word for word in doc.lower().split() if word not in stop_words] for doc in [search_query]]
    corpus1 = [id2word.doc2bow(t) for t in texts1]
    result = lda_model.get_document_topics(corpus1)
    LDA_topic = max(result[0], key=lambda x: x[1])[0]
    return LDA_topic

In [305]:
reference_vector = topic_match('Cirrus Parachute First Responders')
topic_match = [key for key, value in topic_dict.items() if value == reference_vector]
reply_match_search = forum.iloc[topic_match]

In [306]:
reference_vector, lda_model.print_topics(num_words = 10)[reference_vector][1]

(13,
 '0.100*"pilot" + 0.034*"fly" + 0.030*"training" + 0.024*"accident" + 0.022*"cap" + 0.021*"airplane" + 0.020*"cirrus" + 0.017*"flight" + 0.017*"plane" + 0.014*"safety"')

In [307]:
print (reply_match_search['FormattedBody'].head(10))

557     In reply to:Even if CO doesn't turn out to be ...
753     Same battery your BIOS runs on with yr compute...
775     In reply to:MikeMaybe not carbon MONoxide (CO)...
881     Thanks for the update. Do you recall the total...
1346    Jeff,I have a separate XM antenna. I thought y...
1382    In reply to: Sounds like she (yes, A&P is a sh...
2341    In reply to:And who cares - we're all PIC it's...
2606    UPDATED list as of June 25th for the Catalina ...
2820    Put a K&N air filter on a 2004 G2 at my last a...
2915    Larry,My post was triggered by your King Air c...
Name: FormattedBody, dtype: object


__Match replies with searching terms by comparing keywords vector and reference vector__

In [308]:
r = re.compile(r'([.0-9]+)\*\"([a-zA-Z]+)\"')
keyword_vector = {}
for i in range(20):
    keyword_vector[i] = {}
    word_weight = r.findall(lda_model.print_topics(num_words = 10)[i][1])
    for weight, word in word_weight:
        keyword_vector[i][word] = weight

In [309]:
# Compare query to each leaf node, if the partial match score is higher than the threshold, then return a combination vector
# that contains all words in those leaf nodes.
def fuzzywuzzy_match(query,threshold):
    
    # Generate a list contains all leaf nodes, each sublist contains all words in that leaf node.
    leaf_node = list()
    for item in topic_vector:
        if len(topic_vector[item]) == 1:
            leaf_node.append(topic_vector[item][0].split())
            
    reference_vector = list()
    
    for node in leaf_node:
        query_len = len(query)
        node_len = len(node)
        
        if query_len >= node_len: # If query is longer
            for i in range(node_len):
                score = fuzz.partial_ratio(node[i],query)
                if score > threshold:
                    reference_vector.append(node) 
                    
        else: # If leaf node is longer
            for j in range(query_len):
                score = fuzz.partial_ratio(query[j],node)
                if score > threshold:
                    reference_vector.append(node)
    
    reference_vector = list(list(i) for i in set(map(tuple, reference_vector))) # Deduplication
    
    return reference_vector

In [310]:
threshold = 80
query = "Cirrus Parachute First Responders"
reference_vector = fuzzywuzzy_match(query,threshold)
reference_vector_sim = []
for vectors in reference_vector:
    for words in vectors:
        reference_vector_sim.append(words.lower())
        
reference_vector_sim = list(set(reference_vector_sim))

In [311]:
temp = 0
for i in range(19):
    match = set(keyword_vector[i].keys()) & set(reference_vector_sim)
    if len(match) >= 1:
        weight = keyword_vector[i][list(match)[0]]
        if float(weight) >= float(temp):
            temp = weight
            relevant_topic = i

In [312]:
topic_match = [key for key, value in topic_dict.items() if value == relevant_topic]
reply_match_search = forum.iloc[topic_match]

In [313]:
relevant_topic, lda_model.print_topics(num_words = 10)[relevant_topic][1]

(0,
 '0.026*"system" + 0.022*"work" + 0.022*"update" + 0.019*"use" + 0.018*"mfd" + 0.018*"datum" + 0.018*"display" + 0.017*"software" + 0.014*"avidyne" + 0.013*"avionic"')

In [314]:
print (reply_match_search['FormattedBody'].head(10))

2     If I come over Tuesday do I get to play Igor t...
17    In reply to:Usual disclaimers:For display use ...
22    Set going out today to Mike Danko.BTW, the rea...
24    Fast Eddie,I made two of these up when you fir...
35    Going out in today's mail:Donald Thompson, Jr....
51    Sets going out today to:Andrew BarnardGeorge G...
55    Sets going out in today's mail to:Lars Helgese...
60    Know you have many responses but would gladly ...
62              Thanks Ed, I'm sending you an envelope.
77    I don't have a shipping weight, but the post o...
Name: FormattedBody, dtype: object


__Save the model__

In [90]:
# Save model to disk.
temp_file = datapath("lda.model")
lda_model.save(temp_file)

# Load lda.model
lda = gensim.models.ldamodel.LdaModel.load(temp_file)

In [22]:
# pd.DataFrame(lda_model.print_topics(num_words = 10), columns = ['Topic', 'Keywords']).to_csv('LDA_vector.csv')

__Normalization__

In [10]:
def Normalization(col):
    nomolized_col = (df[col] - df[col].mean()) / df.loc[:, col].std()
    return nomolized_col

df['RecencyRate'] = Normalization('RecencyRate')
df['AuthorScore'] = Normalization('AuthorScore')

In [11]:
df.head()

,Date,Title,Content,Author,Like,Comment,Resource,SourceScore,RecencyRate,AuthorScore
0,2015-01-01,President's Column,JANUARY FEBRUARY 20154CIRRUS PILOTAs this is b...,NaN,0,0,Magazine,3,-0.465959,-1.209951
1,2015-01-01,COPA News,JANUARY FEBRUARY 20156CIRRUS PILOTWith this is...,NaN,0,0,Magazine,3,-0.465959,-1.209951
2,2015-01-01,Regional News,JANUARY FEBRUARY 201512CIRRUS PILOTby GIL WILL...,GIL WILLIAMSON,0,0,Magazine,3,-0.465959,0.211050
3,2015-01-01,Cirrus Perspective,JANUARY FEBRUARY 201518CIRRUS PILOTIts hard to...,NaN,0,0,Magazine,3,-0.465959,-1.209951
4,2015-01-01,Member Spotlight,JANUARY FEBRUARY 201522CIRRUS PILOTCirrus Pilo...,KIM BLONIGEN,0,0,Magazine,3,-0.465959,0.211050


__Input Searching query__

For Search Results Ranking

- "Cirrus Parachute First Responders"
- "Different between AATD and FTD"
- "Activate vector to final"
- "Turbo normalize vs turbo charge"
- "Shooting ILS approach"

In [19]:
def extract_keywords(search_query):
    '''
    Get the most important keyword from searching query 
    '''
    stop_words = stopwords.words('english')
    stop_words.append('vs')
    r = Rake(stopwords = stop_words)
    
    r.extract_keywords_from_text(search_query)
    rank = r.get_ranked_phrases_with_scores()
    itemMaxValue = max(rank, key=lambda x: x[0])[0]
    
    # Find the most important keyword
    listOfKeys = list()
    for value, key in rank:
        if value == itemMaxValue:
            listOfKeys.append(key.lower())
    
    return listOfKeys

In [20]:
# search_query = ["Cirrus Parachute First Responders", "Difference between AATD and FTD", "Activate vector to final", "Turbo normalize vs turbo charge", "Shooting ILS approach"]
# extract_keywords("Difference between AATD and FTD")

In [118]:
taxonomy_terms = [tree['Title'] for tree in taxonomy_data['data'][1:]]

topic_vector = {}
for i in range(len(taxonomy_terms)):
    topic_info = taxonomy_data['data'][1:][i] # Sub level of all terms other than COPA
    title = re.findall(r'Title\': \'(.*?)\'', str(topic_info)) # Extract all children titles under specific term
    if len(title) > 1: # If the term is among the bottom level, then just assign itself to the term
        topic_vector[taxonomy_terms[i].lower()] = title[1:]
    else:
        topic_vector[taxonomy_terms[i].lower()] = [taxonomy_terms[i]]

Think about which fuzzywuzzy method we should use:
For each vector in topic_vector: which level we should match to.
What kind of vector we want to get in the end? 
Current version: Combine all the match vectors together.


In [119]:
threshold = 80  # Maybe adjusted to improve performance

In [125]:
# Compare query to each leaf node, if the partial match score is higher than the threshold, then return a combination vector
# that contains all words in those leaf nodes.
def fuzzywuzzy_match(query,threshold):
    
    # Generate a list contains all leaf nodes, each sublist contains all words in that leaf node.
    leaf_node = list()
    for item in topic_vector:
        if len(topic_vector[item]) == 1:
            leaf_node.append(topic_vector[item][0].split())
            
    reference_vector = list()
    
    for node in leaf_node:
        query_len = len(query)
        node_len = len(node)
        
        if query_len >= node_len: # If query is longer
            for i in range(node_len):
                score = fuzz.partial_ratio(node[i],query)
                if score > threshold:
                    reference_vector.append(node) 
                    
        else: # If leaf node is longer
            for j in range(query_len):
                score = fuzz.partial_ratio(query[j],node)
                if score > threshold:
                    reference_vector.append(node)
    
    reference_vector = list(list(i) for i in set(map(tuple, reference_vector))) # Deduplication
    
    return reference_vector

In [183]:
query = "Cirrus Parachute First Responders"
fuzzywuzzy_match(query,threshold)

[['Parachute'],
 ['Cirrus', 'Airframe', 'Parachute', 'System'],
 ['Cirrus', 'Approach'],
 ['Parachute', 'Disreef'],
 ['Over', 'a', 'Runaway'],
 ['Reefed', 'Parachute'],
 ['Rocket', 'and', 'Parachute', 'Extraction'],
 ['Not', 'Over', 'a', 'Runaway']]

In [97]:
def exact_match(search_query):
    '''
    The searching query can be exact matched by taxonomy tree.
    '''
    if search_query in taxonomy_terms:
        taxonomy_vector = topic_vector[search_query.lower()]
    return taxonomy_vector

In [101]:
# exact_match('Tire Brands')

['Goodyear', 'Michelin', 'Aero Classic', 'STA']

In [99]:
# Create a vector that stores all the phrases(more than one word) that should be 
# token as a whole.
def generate_phrases_tokenizer(topic_vector): # Return a list of tuples containing phrases.
    all_multiword_tokens = list()
    for item in topic_vector: 
        for phrase in topic_vector[item]:
            phrase = phrase.replace('(','') # Eliminate '(' and ')' in the phrase
            phrase = phrase.replace(')','')
           #phrase = re.sub(r'( \(.*\))','',phrase) #exclude all contents in '()'
            phrase_lower = phrase.lower() # Convert all characters into lower case.
            #phrase_lower.replace('-', ' ')
            word_lst = phrase_lower.split()
            if len(word_lst) >= 2: 
                word_tuple = tuple(word_lst)
                all_multiword_tokens.append(word_tuple)  
    all_multiword_tokens = list(set(all_multiword_tokens)) # Deduplication
    tokenizer = MWETokenizer(all_multiword_tokens,separator=' ')
    return tokenizer

In [100]:
def any_match(search_query,tokenizer):
    '''
    Any words in searching query can be exact matched by taxonomy tree.
    '''
    # Split input string into words or phrases according to taxonomy tree.
    words = tokenizer.tokenize(search_query.split())
    taxonomy_vector = []
    for word in words:
        try:
            taxonomy_vector.append(topic_vector[word])
        except:
            continue
    return taxonomy_vector

In [84]:
phrase_tokenizer = generate_phrases_tokenizer(topic_vector)
any_match("Turbo normalize vs turbo charge", phrase_tokenizer)

[['Turbo']]

In [191]:
def keyword_match(keywords):
    '''
    The most important keyword of searching query can be matched by taxonomy tree.
    '''
    taxonomy_vector = []
    for keyword in keywords:
        try:
            taxonomy_vector.append(topic_vector[keyword])
        except:
            continue
    return taxonomy_vector

In [229]:
# keywords = extract_keywords("Engine doesn't work.")
# keyword_match(keywords), keywords

([['Engine']], ['work', 'engine'])

In [95]:
def topic_match(search_query):
    '''
    Extract the topic of searching query by LDA model trained by forum posts.
    '''
    texts1 = [[word for word in doc.lower().split() if word not in stop_words] for doc in [search_query]]
    corpus1 = [id2word.doc2bow(t) for t in texts1]
    result = lda_model.get_document_topics(corpus1)
    LDA_topic = max(result[0], key=lambda x: x[1])[0]
    return LDA_topic

In [82]:
# topic_match('Turbo normalize vs turbo charge'), lda_model.print_topics(num_words = 10)[12]

(12,
 (12,
  '0.101*"altitude" + 0.083*"high" + 0.065*"climb" + 0.041*"low" + 0.034*"performance" + 0.032*"gun" + 0.030*"turbo" + 0.028*"level" + 0.028*"foot" + 0.027*"rate"'))